### Adjust Word Group Repeat With Youtube Link

In [34]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from nltk import ngrams
from functools import reduce
from pathlib import Path
import shutil

In [35]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [81]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 10000  # 28 native word end index

# file extention
file_ext = "Fivegram"

In [82]:
def df_col_value_join_comma(df, df_columns_list):
    '''
    df_col_value_join_comma(df_test, ["video_id","start_time","end_time"])\n
    function used for selected column value join with comma in one row
    '''
    column_value_list = []
    for column in df_columns_list:
        list_var = df[f"{column}"].to_list()
        list_var_string = [str(x) for x in list_var] 
        list_var_join = ",".join(list_var_string)
        column_value_list.append(list_var_join)

    df_result = pd.DataFrame([[column_value_list[0],column_value_list[1],column_value_list[2]]], columns=df_columns_list)

    return df_result

In [83]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Talk Time/Talk Time 4/Result/3-Adjust Word Group Repeat With Youtube Link"

Path(path).mkdir(parents=True, exist_ok=True)

##### Link Count

In [84]:
path_folder = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Talk Time/Talk Time 4/Result/2-Find Word Group Youtube Link"
file = f"{lang_folder.capitalize()}_{file_ext}_With_{word_end}_Word_Selected_Manuel.xlsx"
sheet = "Sheet1"  # Sheet1

In [85]:
df_link_selected_manuel = pd.read_excel(f"{path_folder}/{file}", sheet_name=f"{sheet}")
df_link_selected_manuel

,search_string,start_time,end_time,video_id
0,başka bir şey var mı,662,664,8ek8uhoEQ9w
1,başka bir şey var mı,8,10,EtrRFFyvFKI
2,başka bir şey var mı,1392,1394,ryJ7MtDof9Y
3,başka bir şey var mı,1266,1268,MvUL_IHyESU
4,başka bir şey var mı,6230,6232,Oz7FkuxsmJU
5,başka bir şey var mı,432,434,UtcBPDTBiko
6,bu da ne demek oluyor,1482,1485,o_bTQCDwkYA
7,bu da ne demek oluyor,448,451,NIcaM2L_6io
8,bu da ne demek oluyor,349,351,VAclPwWc96U
9,bu da ne demek oluyor,257,260,UMP4zpOyj00


In [86]:
df_link_selected_manuel["video_url"] = "https://www.youtube.com/watch?v="+df_link_selected_manuel['video_id'].map(str)+"&t="+df_link_selected_manuel['start_time'].map(str)+"s"
df_link_selected_manuel

,search_string,start_time,end_time,video_id,video_url
0,başka bir şey var mı,662,664,8ek8uhoEQ9w,https://www.youtube.com/watch?v=8ek8uhoEQ9w&t=...
1,başka bir şey var mı,8,10,EtrRFFyvFKI,https://www.youtube.com/watch?v=EtrRFFyvFKI&t=8s
2,başka bir şey var mı,1392,1394,ryJ7MtDof9Y,https://www.youtube.com/watch?v=ryJ7MtDof9Y&t=...
3,başka bir şey var mı,1266,1268,MvUL_IHyESU,https://www.youtube.com/watch?v=MvUL_IHyESU&t=...
4,başka bir şey var mı,6230,6232,Oz7FkuxsmJU,https://www.youtube.com/watch?v=Oz7FkuxsmJU&t=...
5,başka bir şey var mı,432,434,UtcBPDTBiko,https://www.youtube.com/watch?v=UtcBPDTBiko&t=...
6,bu da ne demek oluyor,1482,1485,o_bTQCDwkYA,https://www.youtube.com/watch?v=o_bTQCDwkYA&t=...
7,bu da ne demek oluyor,448,451,NIcaM2L_6io,https://www.youtube.com/watch?v=NIcaM2L_6io&t=...
8,bu da ne demek oluyor,349,351,VAclPwWc96U,https://www.youtube.com/watch?v=VAclPwWc96U&t=...
9,bu da ne demek oluyor,257,260,UMP4zpOyj00,https://www.youtube.com/watch?v=UMP4zpOyj00&t=...


In [87]:
df_count = df_link_selected_manuel.groupby(["search_string"])[["search_string"]].count()
df_count.rename(columns={"search_string":"count"}, inplace=True)
df_count.sort_values(by="count", ascending=False, inplace=True)
df_count.reset_index(inplace=True)
df_count

,search_string,count
0,bu da ne demek oluyor,7
1,başka bir şey var mı,6
2,o kadar da kötü değil,5
3,daha iyi bir fikrim var,4
4,sana bir şey sorabilir miyim,3
5,yapabileceğim bir şey var mı,3
6,sence de öyle değil mi,2
7,söylemem gereken bir şey var,2
8,bu da ne demek şimdi,1
9,sana bir şey göstermek istiyorum,1


In [88]:
df_count = df_count[df_count["count"] > 2]

In [89]:
df_count_merge = pd.merge(df_link_selected_manuel, df_count, how="inner",on= "search_string")
df_count_merge.sort_values(by=["count","search_string"], ascending=False, inplace=True)
df_count_merge.reset_index(drop=True, inplace=True)
df_count_merge

,search_string,start_time,end_time,video_id,video_url,count
0,bu da ne demek oluyor,1482,1485,o_bTQCDwkYA,https://www.youtube.com/watch?v=o_bTQCDwkYA&t=...,7
1,bu da ne demek oluyor,448,451,NIcaM2L_6io,https://www.youtube.com/watch?v=NIcaM2L_6io&t=...,7
2,bu da ne demek oluyor,349,351,VAclPwWc96U,https://www.youtube.com/watch?v=VAclPwWc96U&t=...,7
3,bu da ne demek oluyor,257,260,UMP4zpOyj00,https://www.youtube.com/watch?v=UMP4zpOyj00&t=...,7
4,bu da ne demek oluyor,447,449,x50BKmNnl8k,https://www.youtube.com/watch?v=x50BKmNnl8k&t=...,7
5,bu da ne demek oluyor,257,259,fCAke_9xf1g,https://www.youtube.com/watch?v=fCAke_9xf1g&t=...,7
6,bu da ne demek oluyor,104,106,DUKsBcVimrg,https://www.youtube.com/watch?v=DUKsBcVimrg&t=...,7
7,başka bir şey var mı,662,664,8ek8uhoEQ9w,https://www.youtube.com/watch?v=8ek8uhoEQ9w&t=...,6
8,başka bir şey var mı,8,10,EtrRFFyvFKI,https://www.youtube.com/watch?v=EtrRFFyvFKI&t=8s,6
9,başka bir şey var mı,1392,1394,ryJ7MtDof9Y,https://www.youtube.com/watch?v=ryJ7MtDof9Y&t=...,6


In [90]:
df_count.to_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Talk Time/Talk Time 4/Data/Deployment/Talk Time 4 {file_ext} Video List.xlsx", index=False)

In [91]:
df_count_merge.to_excel(f"{path_folder}/{lang_folder.capitalize()}_{file_ext}_With_{word_end}_Word_Selected_Count_Manuel.xlsx", index=False)

##### Link Repeat

In [92]:
path_folder_file2 = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Talk Time/Talk Time 4/Result/2-Find Word Group Youtube Link/{lang_folder.capitalize()}_{file_ext}_With_{word_end}_Word_Selected_Count_Manuel.xlsx"
sheet2 = "Sheet1"  # Sheet1

In [93]:
df_word_group_time_loc = pd.read_excel(f"{path_folder_file2}", sheet_name=f"{sheet2}")
#df_word_group_time_loc = pd.read_excel(f"Twogram Threegram Youtube Link Selected_Manuel.xlsx")   
df_word_group_time_loc

,search_string,start_time,end_time,video_id,video_url,count
0,bu da ne demek oluyor,1482,1485,o_bTQCDwkYA,https://www.youtube.com/watch?v=o_bTQCDwkYA&t=...,7
1,bu da ne demek oluyor,448,451,NIcaM2L_6io,https://www.youtube.com/watch?v=NIcaM2L_6io&t=...,7
2,bu da ne demek oluyor,349,351,VAclPwWc96U,https://www.youtube.com/watch?v=VAclPwWc96U&t=...,7
3,bu da ne demek oluyor,257,260,UMP4zpOyj00,https://www.youtube.com/watch?v=UMP4zpOyj00&t=...,7
4,bu da ne demek oluyor,447,449,x50BKmNnl8k,https://www.youtube.com/watch?v=x50BKmNnl8k&t=...,7
5,bu da ne demek oluyor,257,259,fCAke_9xf1g,https://www.youtube.com/watch?v=fCAke_9xf1g&t=...,7
6,bu da ne demek oluyor,104,106,DUKsBcVimrg,https://www.youtube.com/watch?v=DUKsBcVimrg&t=...,7
7,başka bir şey var mı,662,664,8ek8uhoEQ9w,https://www.youtube.com/watch?v=8ek8uhoEQ9w&t=...,6
8,başka bir şey var mı,8,10,EtrRFFyvFKI,https://www.youtube.com/watch?v=EtrRFFyvFKI&t=8s,6
9,başka bir şey var mı,1392,1394,ryJ7MtDof9Y,https://www.youtube.com/watch?v=ryJ7MtDof9Y&t=...,6


In [94]:
search_list = df_word_group_time_loc.iloc[:,0].unique()
len(search_list)

6

In [95]:
# other option 
# mUf7VNqChac =>  black screen
# 0_CDMstFg7M => 10sn
# bj1JRuyYeco => 20sn
# cElhIDdGz7M => screensaver
default_video_id = "Q-8I-uMUMYA"
df_link_default = pd.DataFrame(data=[["repeat",5,7,"repeat_again",f"{default_video_id}",f"https://www.youtube.com/watch?v={default_video_id}&t=0s"]], columns=["search_string","start_time","end_time","sentence","video_id","video_url"])
df_link_default

,search_string,start_time,end_time,sentence,video_id,video_url
0,repeat,5,7,repeat_again,Q-8I-uMUMYA,https://www.youtube.com/watch?v=Q-8I-uMUMYA&t=0s


In [96]:
#(df_result_repeat["end_time"].sum()-df_result_repeat["start_time"].sum())/60

In [102]:
# Each word group result convert to join result in one row
df_all_join_result = pd.DataFrame()
seq_num = 0  # option
for word_group in search_list:
    df_result_repeat = pd.DataFrame()
    # for repeat 1
    df_word_group_search_repeat1 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group]
    # for repeat 2
    try:
        df_word_group_search_repeat2 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group].sample(1)
    except:
        df_word_group_search_repeat2 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group].tail(1)
    df_word_group_search_repeat2.reset_index(drop=True, inplace=True)   
    # repeat block
    for i in range(len(df_word_group_search_repeat1)):        
        df_result_repeat = pd.concat([df_result_repeat,df_word_group_search_repeat1.iloc[[i,]]], axis=0)

    try:
        df_link_default_var = df_link_default
        word_time_diff_var = df_word_group_search_repeat2.loc[0 ,"end_time"] - df_word_group_search_repeat2.loc[0 ,"start_time"]
        if word_time_diff_var < 4:
            word_time_diff_var2 = 3
        else:
            word_time_diff_var2 = word_time_diff_var+1.0

        df_link_default_var.loc[0,"end_time"] = df_link_default_var.loc[0,"start_time"] + word_time_diff_var2
    except:
        pass

    for j in range(3):
        df_result_repeat = pd.concat([df_result_repeat,df_word_group_search_repeat2], axis=0)
        df_result_repeat = pd.concat([df_result_repeat,df_link_default_var], axis=0)
        df_result_repeat.reset_index(drop=True, inplace=True)

    df_result_repeat_var = df_col_value_join_comma(df_result_repeat, ["video_id","start_time","end_time"]) 
    df_result_repeat_var.loc[0,["search_string"]] = f"{word_group}"  # option for search string
    df_result_repeat_var.loc[0,["count"]] = df_result_repeat.loc[0,"count"] # option for count num     
    df_all_join_result = pd.concat([df_all_join_result,df_result_repeat_var], axis=0)

df_all_join_result.reset_index(drop=True, inplace=True)
df_all_join_result["web_format"] = f"[play_lms_videos video_id="+df_all_join_result["video_id"].map(str)+" start_\
time="+df_all_join_result["start_time"].map(str)+" end_time="+df_all_join_result["end_time"].map(str)+"]"
df_all_join_result.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Youtube_Talk_Time_Join_{file_ext}.xlsx", index=False)


In [103]:
df_all_join_result

,video_id,start_time,end_time,search_string,count,web_format
0,"o_bTQCDwkYA,NIcaM2L_6io,VAclPwWc96U,UMP4zpOyj0...","1482,448,349,257,447,257,104,104,5,104,5,104,5","1485,451,351,260,449,259,106,106,8,106,8,106,8",bu da ne demek oluyor,7.0,"[play_lms_videos video_id=o_bTQCDwkYA,NIcaM2L_..."
1,"8ek8uhoEQ9w,EtrRFFyvFKI,ryJ7MtDof9Y,MvUL_IHyES...","662,8,1392,1266,6230,432,1392,5,1392,5,1392,5","664,10,1394,1268,6232,434,1394,8,1394,8,1394,8",başka bir şey var mı,6.0,"[play_lms_videos video_id=8ek8uhoEQ9w,EtrRFFyv..."
2,"pWZnIAh9MXo,XNBOk8QO8I0,LYKmAs0fHtY,WluihGTzAz...","2756,6110,224,161,310,2756,5,2756,5,2756,5","2758,6112,226,163,312,2758,8,2758,8,2758,8",o kadar da kötü değil,5.0,"[play_lms_videos video_id=pWZnIAh9MXo,XNBOk8QO..."
3,"7G29Odeyqpk,LMHUJ60Fsho,Sr17L23OkV0,Uo5B3AGNE_...","7952,858,242,118,858,5,858,5,858,5","7955,861,244,120,861,8,861,8,861,8",daha iyi bir fikrim var,4.0,"[play_lms_videos video_id=7G29Odeyqpk,LMHUJ60F..."
4,"qe8TjfIqZ8Y,cslXqrcH5qg,yXHOeK8xMYk,qe8TjfIqZ8...","62,6502,926,62,5,62,5,62,5","64,6504,928,64,8,64,8,64,8",yapabileceğim bir şey var mı,3.0,"[play_lms_videos video_id=qe8TjfIqZ8Y,cslXqrcH..."
5,"2MFCSF60pLg,MWMmHcPKvfs,g81mNCrBnO4,g81mNCrBnO...","37,5980,4115,4115,5,4115,5,4115,5","41,5983,4117,4117,8,4117,8,4117,8",sana bir şey sorabilir miyim,3.0,"[play_lms_videos video_id=2MFCSF60pLg,MWMmHcPK..."


#### Copy Move And Delete

In [99]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{word_end}_Word_Youtube_Talk_Time_Join_{file_ext}.xlsx")
output_file

['Turkish_5000_Word_Youtube_Talk_Time_Join_Fivegram.xlsx']

In [100]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [101]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass